In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib
from urllib.request import urlopen
import re
from tqdm import tqdm
import pickle
import concurrent
import numpy as np
from pymongo import MongoClient

In [13]:
def parse_word(word):
    html = urlopen("https://forvo.com/word/" + urllib.parse.quote(word.encode("utf-8"))).read().decode()

    root = BeautifulSoup(html, "html.parser")

    answer = {}
    answer["word"] = word
    categories = []

    for a in root.find("div", {"class": "categories"}).findAll("a"):
        categories.append((a.text, a["href"]))

    answer["metadata"] = {
        "added": root.find("div", {"class": "metadata"}).text.split("\n")[1].split(" ")[-1], 
                          "num_listenings": root.find("div", {"class": "metadata"}).find("span").text
    }
    try:
        answer["langs"] = [i.text for i in root.find("nav", {"class": "nav_langs"}).findAll("li")]
    except AttributeError:
        pass
    pronounce = []

    for article in root.findAll("article"):
        for li in article.findAll("li"):
            tmp = {}
            try:
                tmp["author"] = li.find("a", {"class": "uLink"}).text
                tmp["votes"] = li.find("span", {"class": "num_votes"}).text
                tmp["href"] = li.find("p", {"class": "download"}).find("a")["href"]
            except:
                continue

            pronounce.append(tmp)

    answer["pronounces"] = pronounce
    
    return answer

In [ ]:
tags = []

for i in tqdm(range(6429)):
    url = "https://forvo.com/tags/all/by-popularity/page-{0}/".format(i)
    html = urlopen(url).read().decode()

    root = BeautifulSoup(html, "html.parser")
    
    for li in root.find("section", {"class": "main_section"}).findAll("li"):
        a = li.find("a")
        try:
            if "/tag/" in a["href"]:
                tags.append((a["href"], a.text))
        except TypeError:
            pass

In [4]:
#pickle.dump(tags, open("tags.pck","wb"))
tags = pickle.load(open("tags.pck","rb"))

In [5]:
global_htmls = []

In [20]:
def get_ans(das):
    url = das + "alphabetically/"
    html = urlopen(url).read().decode()
    root = BeautifulSoup(html, "html.parser")
    all_aph = []
    for li in root.find("nav", {"class": "alphabet"}).findAll("li"):
        all_aph.append(url[:-24] + li.find("a")['href'])

    def get_words(root):
        for li in root.findAll("ul")[-4].findAll("li"):
            try:
                word = li.find("a").text
            except AttributeError:
                return
            word = word.replace(" pronunciation", "").replace(" ","_")
            try:
                tmp_client = MongoClient()
                tmp_db = tmp_client.voice
                tmp_db.word.insert_one(parse_word(word))
            except urllib.error.HTTPError:
                pass
        
        if root.find("a", {"rel": "next"}) != None:
            get_words(BeautifulSoup(urlopen(root.find("a", {"rel": "next"})["href"]).read().decode(), "html.parser"))
    
    for url in tqdm(all_aph):
        get_words(BeautifulSoup(urlopen(url).read().decode(), "html.parser"))

In [ ]:
for i in tqdm([i[0] for i in tags][2:]):
    get_ans(i)

  4%|▎         | 1/28 [00:00<00:03,  7.52it/s]

In [ ]:
len()